# Generate Aggregate Data for Peer Reviewers
Generates a .csv file called reviewer_agg_stats.csv in /data/ with aggregate reviewer stats

In [1]:
import pandas as pd
import os

try:
    import nltk
except ImportError as error:
    print(f'{error}\nInstalling nltk')
    !mamba install nltk --quiet --yes
    nltk.download("punkt")
    
from numpy import mean
import re

os.chdir('/home/jovyan/shared/2020_06_10_bad_reviewer')

In [3]:
data_view = pd.read_csv("bin/reviews_df.csv", index_col=[0])
#peer_skips = pd.read_csv("data/peer_skips.csv", escapechar='\\', on_bad_lines='warn', index_col=[0])
len(data_view)

1084657

In [3]:
data_view.drop(list(range(755643, 755650)), axis=0, inplace=True)
data_view.drop(list(range(1025262, 1025269)), axis=0, inplace=True)

## Create Columns for Types, Tokens and Type/Token Ratio
### The first cell creates a dataframe with average tokens, types and ttr for responses by reviewers

In [4]:
# Calculate average ttr for individual responses
data_view['tokenized_response'] = data_view.apply(
    lambda row: [token for token in nltk.word_tokenize(str(row.peer_review_part_free_response_text).lower()) if re.search('[a-zA-Z]', token)], axis=1)
data_view['response_tokens'] = data_view.apply(lambda row: len(row.tokenized_response), axis=1)
data_view['response_types'] = data_view.apply(lambda row: len(set(row.tokenized_response)), axis=1)
data_view['response_ttr'] = data_view['response_types']/data_view['response_tokens']
data_view['num_responses'] = data_view.apply(lambda row: 1 if row.response_tokens > 0 else 0, axis=1)
# Aggregate the data
agg_responses = data_view.groupby('uva_peer_assignments_user_id').agg({'response_tokens':'mean', 'response_types':'mean', 'response_ttr':'mean', 'num_responses':'sum'}).reset_index()
agg_responses

,uva_peer_assignments_user_id,response_tokens,response_types,response_ttr,num_responses
0,0000e5af02da0c7575b3ebd346b55b29f959e90f,17.476190,15.142857,0.914908,21
1,0002c0f31f5c8456bd360dfcd089a64f444e2de0,15.380952,14.238095,0.934807,21
2,00030b378ea62d60a177113b7854eb26cc29e1a9,3.476190,3.476190,1.000000,21
3,000458f7d47a0b6414f9146258829170ae3ed6a9,2.000000,2.000000,1.000000,21
4,00069909160c6bcd9836cdb23e35b1fdaf56d0c3,1.666667,1.666667,1.000000,21
...,...,...,...,...,...
41238,fff56a1858c62540bd76bad23db07a2fbfa963fe,12.428571,11.285714,0.949896,21
41239,fff7364558963fb9fa218f2fa08d5d2767992207,7.190476,7.000000,0.986317,21
41240,fff883a717a37b5699e5105a96d61dc10812cabe,1.000000,1.000000,1.000000,28
41241,fffa6add42713b8b1e2a5158616f780997bbda49,6.904762,6.761905,0.993266,21


### This cell creates a dataframe with total types, tokens, and TTR for all responses by each reviewer

In [15]:
# Calculate total TTR for all responses by each reviewer
combined_responses = data_view.groupby(['uva_peer_assignments_user_id']).agg({'tokenized_response':['sum']}).reset_index()
combined_responses.columns = ['uva_peer_assignments_user_id', 'tokenized']
combined_responses['total_tokens'] = combined_responses.apply(lambda row: len(row.tokenized), axis=1)
combined_responses['total_types'] = combined_responses.apply(lambda row: len(set(row.tokenized)), axis=1)
combined_responses['total_ttr'] = combined_responses['total_types']/combined_responses['total_tokens']
combined_responses.pop('tokenized')
combined_responses

,uva_peer_assignments_user_id,total_tokens,total_types,total_ttr
0,0000e5af02da0c7575b3ebd346b55b29f959e90f,367,180,0.490463
1,0002c0f31f5c8456bd360dfcd089a64f444e2de0,323,150,0.464396
2,00030b378ea62d60a177113b7854eb26cc29e1a9,73,28,0.383562
3,000458f7d47a0b6414f9146258829170ae3ed6a9,42,2,0.047619
4,00069909160c6bcd9836cdb23e35b1fdaf56d0c3,35,11,0.314286
...,...,...,...,...
41238,fff56a1858c62540bd76bad23db07a2fbfa963fe,261,127,0.486590
41239,fff7364558963fb9fa218f2fa08d5d2767992207,151,82,0.543046
41240,fff883a717a37b5699e5105a96d61dc10812cabe,28,1,0.035714
41241,fffa6add42713b8b1e2a5158616f780997bbda49,145,89,0.613793


## Aggregate Data About the Scores
Get data about the number of reviews, the mean and sd criteria scores, and the mean and sd total scores

In [6]:
# Calculate and aggregate data about the scores
import numpy as np
def std(x): return np.std(x)

agg_func_math = {
    'peer_assignment_review_schema_part_option_score':
    ['size', 'mean', std, 'sum']
}
agg_data = data_view.groupby(['uva_peer_assignments_user_id', 'peer_submission_id']).agg(agg_func_math).reset_index()
agg_data.columns = ['uva_peer_assignments_user_id', 'peer_submission_id', 'num_reviews', 'review_mean', 'review_std', 'review_score']
agg_data.num_reviews = agg_data.num_reviews/7
agg_score_data = agg_data.groupby('uva_peer_assignments_user_id').agg({'num_reviews':'sum', 'review_mean':'mean', 'review_std':'mean', 'review_score':['mean', std]}).reset_index()
agg_score_data.columns = ['uva_peer_assignments_user_id', 'num_reviews', 'criteria_mean', 'criteria_sd', 'total_mean', 'total_sd']

In [7]:
agg_score_data[agg_score_data['total_mean'] > 18]
# It looks like these two reviewers rated the same essay twice. I'm going to go back and delete the duplicates from the data_view by index
# 1025262: 1025268
# 755643: 755649
# Better now.

,uva_peer_assignments_user_id,num_reviews,criteria_mean,criteria_sd,total_mean,total_sd


## Get Information about Number of Skips

In [9]:
# Make a list of submissions that were skipped more than twice
skipped_submissions = list(peer_skips['peer_submission_id'])
multiple_skips = []
for submission in set(skipped_submissions):
    if skipped_submissions.count(submission) > 1:
        multiple_skips.append(submission)

# calculate how many solo reviews each reviewer did
data_view_unique = data_view[['peer_submission_id', 'uva_peer_assignments_user_id']].drop_duplicates()
data_view_unique['solo_reviews'] = data_view_unique.apply(lambda row: 1 if row.peer_submission_id in multiple_skips else 0, axis=1)
data_view_unique

agg_skips = data_view_unique.groupby('uva_peer_assignments_user_id').agg({'solo_reviews':'sum'}).reset_index()
agg_skips

,uva_peer_assignments_user_id,solo_reviews
0,0000e5af02da0c7575b3ebd346b55b29f959e90f,0
1,0002c0f31f5c8456bd360dfcd089a64f444e2de0,0
2,00030b378ea62d60a177113b7854eb26cc29e1a9,0
3,000458f7d47a0b6414f9146258829170ae3ed6a9,0
4,00069909160c6bcd9836cdb23e35b1fdaf56d0c3,0
...,...,...
41238,fff56a1858c62540bd76bad23db07a2fbfa963fe,0
41239,fff7364558963fb9fa218f2fa08d5d2767992207,0
41240,fff883a717a37b5699e5105a96d61dc10812cabe,1
41241,fffa6add42713b8b1e2a5158616f780997bbda49,0


## Merge the Dataframes

In [16]:
# Merge the dataframes
reviewer_stats_df = agg_score_data.merge(combined_responses, on='uva_peer_assignments_user_id').merge(agg_responses,on='uva_peer_assignments_user_id').merge(agg_skips,on='uva_peer_assignments_user_id')
reviewer_stats_df['response_ratio'] = reviewer_stats_df['num_responses']/(reviewer_stats_df['num_reviews']*7)
reviewer_stats_df

,uva_peer_assignments_user_id,num_reviews,criteria_mean,criteria_sd,total_mean,total_sd,total_tokens,total_types,total_ttr,response_tokens,response_types,response_ttr,num_responses,solo_reviews,response_ratio
0,0000e5af02da0c7575b3ebd346b55b29f959e90f,3.0,2.111111,0.669706,12.666667,2.624669,367,180,0.490463,17.476190,15.142857,0.914908,21,0,1.0
1,0002c0f31f5c8456bd360dfcd089a64f444e2de0,3.0,2.000000,0.471405,12.000000,2.828427,323,150,0.464396,15.380952,14.238095,0.934807,21,0,1.0
2,00030b378ea62d60a177113b7854eb26cc29e1a9,3.0,2.111111,0.581785,12.666667,3.299832,73,28,0.383562,3.476190,3.476190,1.000000,21,0,1.0
3,000458f7d47a0b6414f9146258829170ae3ed6a9,3.0,3.000000,0.000000,18.000000,0.000000,42,2,0.047619,2.000000,2.000000,1.000000,21,0,1.0
4,00069909160c6bcd9836cdb23e35b1fdaf56d0c3,3.0,2.166667,0.372678,13.000000,0.000000,35,11,0.314286,1.666667,1.666667,1.000000,21,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41238,fff56a1858c62540bd76bad23db07a2fbfa963fe,3.0,1.888889,0.290893,11.333333,4.784233,261,127,0.486590,12.428571,11.285714,0.949896,21,0,1.0
41239,fff7364558963fb9fa218f2fa08d5d2767992207,3.0,2.722222,0.323802,16.333333,1.247219,151,82,0.543046,7.190476,7.000000,0.986317,21,0,1.0
41240,fff883a717a37b5699e5105a96d61dc10812cabe,4.0,3.000000,0.000000,18.000000,0.000000,28,1,0.035714,1.000000,1.000000,1.000000,28,1,1.0
41241,fffa6add42713b8b1e2a5158616f780997bbda49,3.0,2.444444,0.490468,14.666667,0.471405,145,89,0.613793,6.904762,6.761905,0.993266,21,0,1.0


In [17]:
# # Explore the data
for column in reviewer_stats_df:
    print(column)
    print(reviewer_stats_df[column].isnull().sum())
    
reviewer_stats_df[reviewer_stats_df['total_sd'].isnull()].head(50)

uva_peer_assignments_user_id
0
num_reviews
0
criteria_mean
0
criteria_sd
0
total_mean
0
total_sd
0
total_tokens
0
total_types
0
total_ttr
1043
response_tokens
0
response_types
0
response_ttr
1043
num_responses
0
solo_reviews
0
response_ratio
0


,uva_peer_assignments_user_id,num_reviews,criteria_mean,criteria_sd,total_mean,total_sd,total_tokens,total_types,total_ttr,response_tokens,response_types,response_ttr,num_responses,solo_reviews,response_ratio


In [11]:
data_view[data_view['uva_peer_assignments_user_id_reviewer'] == '0d944e198da0d5a52648944d9b77cebbe471f635'].head(50)

,peer_review_id,uva_peer_assignments_user_id_reviewer,peer_review_created_ts,peer_submission_id,peer_assignment_review_schema_part_prompt_score,peer_assignment_review_schema_part_option_score,peer_assignment_review_schema_part_prompt_text_response,peer_review_part_free_response_text,uva_peer_assignments_user_id_author,peer_submission_score,tokenized_response,response_tokens,response_types,response_ttr,num_responses
520919,DrUyOFGtEeuSow58yD2wQw,0d944e198da0d5a52648944d9b77cebbe471f635,2021-01-08 12:28:48.288,MaK2b1GmEeuZ6BIjhn6kyQ,Challenge (score),3.0,Feedback (ungraded):The submission’s challenge...,Very neat and detailed description!,554057484c3238a33f39914d40ae5eefc1f76a9b,16.0,"[very, neat, and, detailed, description]",5,5,1.0,1
520920,DrUyOFGtEeuSow58yD2wQw,0d944e198da0d5a52648944d9b77cebbe471f635,2021-01-08 12:28:48.288,MaK2b1GmEeuZ6BIjhn6kyQ,Selection (score),2.0,Feedback (ungraded):The submission’s tool sele...,Good information! Maybe include more specific ...,554057484c3238a33f39914d40ae5eefc1f76a9b,16.0,"[good, information, maybe, include, more, spec...",9,9,1.0,1
520921,DrUyOFGtEeuSow58yD2wQw,0d944e198da0d5a52648944d9b77cebbe471f635,2021-01-08 12:28:48.288,MaK2b1GmEeuZ6BIjhn6kyQ,Application (score),3.0,Feedback (ungraded):The submission’s tool appl...,Nothing to criticize!,554057484c3238a33f39914d40ae5eefc1f76a9b,16.0,"[nothing, to, criticize]",3,3,1.0,1
520922,DrUyOFGtEeuSow58yD2wQw,0d944e198da0d5a52648944d9b77cebbe471f635,2021-01-08 12:28:48.288,MaK2b1GmEeuZ6BIjhn6kyQ,Insight (score),3.0,Feedback (ungraded):The submission’s insight d...,"Very detailed,",554057484c3238a33f39914d40ae5eefc1f76a9b,16.0,"[very, detailed]",2,2,1.0,1
520923,DrUyOFGtEeuSow58yD2wQw,0d944e198da0d5a52648944d9b77cebbe471f635,2021-01-08 12:28:48.288,MaK2b1GmEeuZ6BIjhn6kyQ,Approach (score),2.0,Feedback (ungraded):The submission’s approach ...,Also indicated negative aspects.,554057484c3238a33f39914d40ae5eefc1f76a9b,16.0,"[also, indicated, negative, aspects]",4,4,1.0,1
520924,DrUyOFGtEeuSow58yD2wQw,0d944e198da0d5a52648944d9b77cebbe471f635,2021-01-08 12:28:48.288,MaK2b1GmEeuZ6BIjhn6kyQ,Organization (score),3.0,Feedback (ungraded):The submission’s overall o...,Easy to read.,554057484c3238a33f39914d40ae5eefc1f76a9b,16.0,"[easy, to, read]",3,3,1.0,1
520925,DrUyOFGtEeuSow58yD2wQw,0d944e198da0d5a52648944d9b77cebbe471f635,2021-01-08 12:28:48.288,MaK2b1GmEeuZ6BIjhn6kyQ,NaN,NaN,Additional Feedback (ungraded): While reviewin...,Inspired by your mindmap! I understand the dif...,554057484c3238a33f39914d40ae5eefc1f76a9b,16.0,"[inspired, by, your, mindmap, i, understand, t...",11,11,1.0,1


## Write to csv

In [18]:
# Write dataframe to csv
reviewer_stats_df.to_csv('results/reviewer_agg_stats.csv')